## How often does the EPA breakdown of teams in a match correctly predict a match win?

EPA stands for Expected Points Added and is a standard for predicting individual robot contributions. In this project, I will use EPA scored for each match played by Team 4500 to create a working range at a 95% confidence interval.

EPA is similar to Elo but without the arbritary units. The downside is that it usually takes a few matches to get the EPA to a reasonably accurate value. As such, I will make two data sets, one with all the data, and another ignoring the first two matches.

In [22]:
# Example API Call

import statbotics

sb = statbotics.Statbotics()
matches = sb.get_matches(4500)

print("Year: " + str(matches[0].get('year')))
print("Match: " + str(matches[0].get('match_number')))
print("red EPA Sum: " + str(matches[0].get('red_epa_sum')))
print("blue EPA Sum: " + str(matches[0].get('blue_epa_sum')))
print("Winner: " + str(matches[0].get('winner')))
print(sb.get_matches(4500, year=2023))

Year: 2013
Match: 2
red EPA Sum: 33.52
blue EPA Sum: 42.24
Winner: blue
[{'key': '2023mosl_qm7', 'year': 2023, 'event': '2023mosl', 'comp_level': 'qm', 'set_number': 1, 'match_number': 7, 'offseason': False, 'status': 'Completed', 'video': 'l_RrvDe8FX8', 'red_1': 8077, 'red_2': 931, 'red_3': 4143, 'red_dq': '', 'red_surrogate': '', 'red_epa_sum': 71.5, 'red_auto_epa_sum': 19.84, 'red_teleop_epa_sum': 33.23, 'red_endgame_epa_sum': 18.42, 'red_rp_1_epa_sum': -0.3591, 'red_rp_2_epa_sum': 0.3876, 'blue_1': 34, 'blue_2': 6744, 'blue_3': 4500, 'blue_dq': '', 'blue_surrogate': '', 'blue_epa_sum': 67.16, 'blue_auto_epa_sum': 18.64, 'blue_teleop_epa_sum': 31.22, 'blue_endgame_epa_sum': 17.3, 'blue_rp_1_epa_sum': -0.396, 'blue_rp_2_epa_sum': 0.3506, 'winner': 'red', 'epa_winner': 'red', 'epa_win_prob': 0.553, 'red_rp_1_prob': 0.0312, 'red_rp_2_prob': 0.3894, 'blue_rp_1_prob': 0.027, 'blue_rp_2_prob': 0.3549, 'playoff': False, 'time': 1678463280, 'predicted_time': 1678463116, 'red_score': 107, 'b

In [13]:
# Make Population

full_population: list[dict] = []
late_population: list[dict] = []
elim_population: list[dict] = []
TEAM_NUMBER: int = 4500
UP_TO_YEAR = 2023

def compile_all_matches():
  for i in sb.get_matches(team=TEAM_NUMBER):
    full_population.append(i)


def compile_later_matches(skip: int = 2):
  rookie_year = sb.get_team_years(TEAM_NUMBER)[0].get('year')
  for year in range(rookie_year, UP_TO_YEAR):
    try:
      year_matches = sb.get_matches(team=TEAM_NUMBER, year=year)
    except:
      continue
    num: int = 0
    for match in year_matches:
      num += 1
      if (num > skip):
        late_population.append(match)

def compile_elimination_matches():
  for match in sb.get_matches(team=TEAM_NUMBER, elims=True):
    elim_population.append(match)

compile_all_matches()
compile_later_matches(2)
compile_elimination_matches()

In [14]:
# isolate EPAs

pop = full_population

team_years = []
team_match_nums = []
team_epas = []
n = 0
for i in pop:
  n+=1
  team_years.append(i.get('year'))
  team_match_nums.append(n)
  if i.get('red_1') == TEAM_NUMBER or i.get('red_2') == TEAM_NUMBER or i.get('red_3') == TEAM_NUMBER:
    team_epas.append(i.get('red_epa_sum'))
  else:
    team_epas.append(i.get('blue_epa_sum'))


### Now to Sample and Gather Proportions

In [25]:
# Sampler

import random

def get_team_pred_vals(i: dict) -> dict:
  if i.get('winner') == 'red':
    return {'isCorrect': i.get('red_epa_sum') > i.get('blue_epa_sum'), 'actual': i.get('red_score')}
  else:
    return {'isCorrect': i.get('blue_epa_sum') > i.get('red_epa_sum'), 'actual': i.get('blue_score')}

def s_samples_of_n_size(s: int, n: int, popl: list[dict]) -> list[(int, float)]:
  l = []

  for sample in range(s):
    tmp_popl = popl.copy()
    num_corr = 0
    for item in range(n):
      index = random.randint(0, len(tmp_popl)-1)
      predicted = get_team_pred_vals(tmp_popl.pop(index))
      if (predicted.get('isCorrect')):
        num_corr += 1
    l.append((sample, float(num_corr)/float(n)))

  return l

In [26]:
print(sb.get_matches(4500)[0])

{'key': '2013mosl_qm2', 'year': 2013, 'event': '2013mosl', 'comp_level': 'qm', 'set_number': 1, 'match_number': 2, 'offseason': False, 'status': 'Completed', 'video': 'IfarexX0y3A', 'red_1': 4331, 'red_2': 2978, 'red_3': 4500, 'red_dq': '', 'red_surrogate': '', 'red_epa_sum': 33.52, 'red_auto_epa_sum': None, 'red_teleop_epa_sum': None, 'red_endgame_epa_sum': None, 'red_rp_1_epa_sum': None, 'red_rp_2_epa_sum': None, 'blue_1': 1182, 'blue_2': 4402, 'blue_3': 3862, 'blue_dq': '', 'blue_surrogate': '', 'blue_epa_sum': 42.24, 'blue_auto_epa_sum': None, 'blue_teleop_epa_sum': None, 'blue_endgame_epa_sum': None, 'blue_rp_1_epa_sum': None, 'blue_rp_2_epa_sum': None, 'winner': 'blue', 'epa_winner': 'blue', 'epa_win_prob': 0.4005, 'red_rp_1_prob': None, 'red_rp_2_prob': None, 'blue_rp_1_prob': None, 'blue_rp_2_prob': None, 'playoff': False, 'time': 1363233602, 'predicted_time': None, 'red_score': 11, 'blue_score': 36, 'red_auto': None, 'red_auto_movement': None, 'red_teleop': None, 'red_endgame'